# Projeto 3#
#### Antonio Fonseca, Daniel Pucciariello e José Bechara.####

## Tema:##
*Prever se um projeto do kickstarter irá ser bem sucedido ou não.*

Técnica: Classificador.

https://www.kaggle.com/kemical/kickstarter-projects


In [1]:
%matplotlib notebook
%reset -f


import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn import preprocessing

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

### Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

In [3]:
data = pd.read_csv('ks-projects-201801.csv', parse_dates=['deadline', 'launched'])
data1 = pd.read_csv('ks-projects-201801.csv', parse_dates=['deadline', 'launched'])

In [4]:
data = data.query('state != "live"')
data_live=data1.query('state == "live"')

In [5]:
data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [6]:
data.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'usd_pledged_real', 'usd_goal_real'],
      dtype='object')

In [7]:
data.category = data.category.astype('category')
data.main_category = data.main_category.astype('category')
data.state = data.state.astype('category')
data.country = data.country.astype('category')

In [8]:
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

In [9]:
data_dummified = dummify(data, "main_category")
data_dummified = dummify(data_dummified, "state")

data_dummified.head()

,ID,name,category,currency,deadline,goal,launched,pledged,backers,country,...,main_category_Music,main_category_Photography,main_category_Publishing,main_category_Technology,main_category_Theater,state_canceled,state_failed,state_successful,state_suspended,state_undefined
0,1000002330,The Songs of Adelaide & Abullah,Poetry,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,0,GB,...,0,0,1,0,0,0,1,0,0,0
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,15,US,...,0,0,0,0,0,0,1,0,0,0
2,1000004038,Where is Hank?,Narrative Film,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,3,US,...,0,0,0,0,0,0,1,0,0,0
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,1,US,...,1,0,0,0,0,0,1,0,0,0
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,14,US,...,0,0,0,0,0,1,0,0,0,0


In [10]:
#quantitativas = data_dummified.loc[:,['goal', 'pledged', 'launched']
quantitativas = data_dummified.loc[:,['goal', 'pledged', 'state_successful', 'main_category_Photography', 'main_category_Publishing','main_category_Music', 'main_category_Technology','main_category_Theater', ]]
quantitativas.head(20)

,goal,pledged,state_successful,main_category_Photography,main_category_Publishing,main_category_Music,main_category_Technology,main_category_Theater
0,1000.0,0.00,0,0,1,0,0,0
1,30000.0,2421.00,0,0,0,0,0,0
2,45000.0,220.00,0,0,0,0,0,0
3,5000.0,1.00,0,0,0,1,0,0
4,19500.0,1283.00,0,0,0,0,0,0
5,50000.0,52375.00,1,0,0,0,0,0
6,1000.0,1205.00,1,0,0,0,0,0
7,25000.0,453.00,0,0,0,0,0,0
8,125000.0,8233.00,0,0,0,0,0,0
9,65000.0,6240.57,0,0,0,0,0,0


In [11]:
# Separaremos as preditoras
#quant_x = quantitativas.copy()
# quant_x.remove('SalePrice')
#quant_x

In [12]:
#y = data_dummified.state_successful == 1

In [13]:
y = quantitativas.state_successful == 1

In [14]:
X = quantitativas.loc[:,['goal', 'pledged']]
print(type(X))
print(X.shape)
print(X[:5])

<class 'pandas.core.frame.DataFrame'>
(375862, 2)
      goal  pledged
0   1000.0      0.0
1  30000.0   2421.0
2  45000.0    220.0
3   5000.0      1.0
4  19500.0   1283.0


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(quantitativas, y, test_size=0.33)

In [16]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((251827, 8), (251827,), (124035, 8), (124035,))

In [24]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', multi_class='auto',max_iter=200)

In [25]:
conta_treinamento = pd.Series(y_train).value_counts()
conta_treinamento

False    161986
True      89841
Name: state_successful, dtype: int64

In [26]:
conta_teste = pd.Series(y_test).value_counts()
conta_teste

False    79920
True     44115
Name: state_successful, dtype: int64

In [27]:
conta_treinamento[True] / conta_treinamento[False]

0.554622004370748

In [28]:
conta_teste[True] / conta_teste[False]

0.5519894894894894

In [29]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
y_pred = model.predict(X_test)

# Calcula acurácia.
acc = accuracy_score(y_test, y_pred)

print('Acurácia: {}'.format(acc))

Acurácia: 1.0
